# Aprendizagem de Máquina - Parte 1


Neste notebook, vamos aprender a criar modelos de **classificação**, também chamamos de *classificadores* ou *reconhecedores*.

## 1 - Instalações Iniciais

Esta célula faz instalações iniciais, no ambiente Colab. Faça assim:
1. Execute as células desta seção.
1. (**IMPORTANTE**) Depois vá no menu `Ambiente de Execução` $\rightarrow$ `Reiniciar ambiente de execução`.
1. Depois pode executar o restante.

In [ ]:
!pip install numpy==1.21.2 scipy scikit-learn==1.0 matplotlib==3.4.3 pandas==1.3.2 importlib-metadata==4.13

**Atenção**: Depois de instalar, reinicie o ambiente!

Vá no menu `Ambiente de Execução -> Reiniciar ambiente de execução`

## 2 - Definições Auxiliares

Não precisa entender o código dentro desta seção. Ele não é fundamental para usar modelos de aprendizagem de máquina.

### 2.1 - Importando Módulos


Primeiro, importamos alguns dos módulos, usados nas definições. Outros serão importantos ao longo deste notebook.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap
import pandas as pd

### 2.2 - Funções Auxiliares

Definimos duas funções auxiliares, de propósito didático: `plot_decision_regions` e `dataset_to_dataframe`.

A primeira delas, definida logo abaixo, exibe o conjunto de exemplos visualmente. Ela assume que cada exemplo tem apenas *dois* atributos de entrada. Eles serão tratados como coordenadas no plano cartesiano e, assim, cada exemplo, será representado como um *ponto*. A classe de cada dado define o formato (geométrico) do ponto.

In [ ]:
# baseada em
def plot_decision_regions(X_train, y_train, classifier, X_test=None, y_test=None, resolution=0.02):
    if X_test is not None:
       X = np.vstack((X_train, X_test))
       y = np.hstack((y_train, y_test))

    # setup marker generator and color map
    markers = ('o', 's', '^', 'v', '<')
    colors = ('red', 'blue', 'lightgreen', 'gray', 'cyan')
    cmap = ListedColormap(colors[:len(np.unique(y))])

    # plot the decision surface
    x1_min, x1_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    x2_min, x2_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx1, xx2 = np.meshgrid(np.arange(x1_min, x1_max, resolution),
                           np.arange(x2_min, x2_max, resolution))
    lab = classifier.predict(np.array([xx1.ravel(), xx2.ravel()]).T)
    lab = lab.reshape(xx1.shape)
    plt.contourf(xx1, xx2, lab, alpha=0.3, cmap=cmap)
    plt.xlim(xx1.min(), xx1.max())
    plt.ylim(xx2.min(), xx2.max())

    # plot class examples
    for idx, cl in enumerate(np.unique(y)):
        plt.scatter(x=X[y == cl, 0],
                    y=X[y == cl, 1],
                    alpha=0.8,
                    c=colors[idx],
                    marker=markers[idx],
                    label=f'Class {cl}',
                    edgecolor='black')

    # highlight test examples
    if X_test is not None:
        # plot all examples
        plt.scatter(X_test[:, 0],
                    X_test[:, 1],
                    c='none',
                    edgecolor='black',
                    alpha=1.0,
                    linewidth=1,
                    marker='o',
                    s=100,
                    label='Test set')

A segunda função recebe um `dataset` de classificação carregado pelo scikit-learn e devolve um dado do tipo `DataFrame`, para ser exibido como uma tabela detalhada (com índices nas linhas, títulos nas colunas), contendo os atributos de entrada e o de saída.

In [ ]:
def dataset_to_dataframe(sklearn_dataset):
    df = pd.DataFrame(sklearn_dataset.data, columns=sklearn_dataset.feature_names)
    df['target'] = sklearn_dataset.target
    df['target_name'] = df['target'].apply( lambda y : sklearn_dataset.target_names[y] )
    return df


## 3 - Carregando e Preparando um Conjunto de Dados

Vamos usar o conjunto **Iris**, que já foi explicado na aula de Scratch. O scikit-learn oferece uma função para carregá-lo diretamente.

In [ ]:
from sklearn import datasets

iris = datasets.load_iris()
print(iris['DESCR'])

Abaixo, convertemos para uma tabela do tipo `DataFrame` (do módulo `pandas`), apenas para visualização.

Não vamos trabalhar com os dados desta forma!

In [ ]:
dataset_to_dataframe(iris)

### 3.1 - Explorando as informações do dataset:

Vamos acessar cada informação a partir da variável `iris`. Abaixo mostramos as informações disponíveis, que são acessadas por certas *chaves* (ingl.: *keys*):

In [ ]:
print("Informações (chaves) do dataset:")
print(iris.keys())

Por exemplo, vamos acessar a chave `data` dentro da variável `iris`. Ela contém todos os exemplos, mas somente com os atributos de entrada. É como uma tabela, mas dada de forma bruta (sem colunas, nem linhas de título, por exemplo).

In [ ]:
print(iris.data)

Os atributos de saída (ou atributos alvos) correspondentes ficam na chave `target` (trad.: alvo).

In [ ]:
print(iris.target)

Os nomes das colunas ficam no atributo `feature_names` (trad.: nomes das características ou nomes dos atributos).

In [ ]:
iris.feature_names

Se passarmos um índice, acessamos uma linha dos dados. Ou seja, um dos exemplos de treinamento (só os atributos de entrada dele). Abaixo, acessamos a primeira linha (índice 0).

In [ ]:
iris.data[0]

Para acessar, no primeiro exemplo (índice 0), o terceiro atributo dele (índice 2), fornecemos os dois índices na ordem `linha,coluna`:

In [ ]:
#outra forma: iris.data[0][2]
iris.data[0,2]

Para acessar não um elemento (linha ou coluna), mas uma sequência deles, ao invés de um valor único de índice, usamos `:` da forma abaixo:
```
     índice_de_início:índice_de_fim(exclusivo)
```

Por exemplo, para acessar as linhas **de 0 a 5** (sem incluir o 5), fazemos assim:

In [ ]:
iris.data[0:5]  # linhas dos indices 0 a 4

E para acessar, na linha 0, as colunas **de 2 a 4** (sem incluir o 4), fazemos assim:

In [ ]:
iris.data[0, 2:4]

Para acessar todas as colunas da linha 0, você ambém pode deixar só `:` sem início nem fim:

In [ ]:
iris.data[0, :]

Explorando as saídas correspondentes exemplos:

In [ ]:
iris.target

Por exemplo, esta é a saída referente ao primeiro exemplo de entrada (que acessamos como `iris.data[0]`):

In [ ]:
iris.target[0]

As classes (saídas/alvos) estão representadas como números de 0 a 2. Mas quer saber o que esses números representam? Vamos acessar a chave `target_names` (nomes dos alvos):

In [ ]:
print(iris.target_names)

### 3.2 - Criando Variáveis de Entrada (x) e Saída (y)

Agora, vamos preparar os dados na forma que vamos usar no treinamento. Vamos usar essas varáveis:
- `x` representa os exemplos de **entradas**
- `y` representa os exemplos de **saídas**

Como entradas (`x`), poderíamos usar todo o `iris.data`. Mas vamos selecionar apenas 2 colunas, porque permite visualizar e permite criar bons modelos.

In [ ]:
# pega todas as linhas dos dados de entrada
# mas somente da coluna de índice 2 até a coluna antes do índice 4
x = iris.data[:, 2:4]

In [ ]:
x

In [ ]:
# pega exatamente todos os valores de saídas
y = iris.target

In [ ]:
y

Abaixo, detalhamos o exemplo do índice 100 (que é a posição 101). Em `x`, vemos os valores dos seus atributos de entradas. Em `y`, vemos a classe de saída.

In [ ]:
x[100]

In [ ]:
y[100]

Os tamanhos de `x` e `y` devem ser iguais e indicam a quantidade de exemplos do conjunto de dados.

In [ ]:
len(x)

In [ ]:
len(y)

### 3.3 - Dividindo em Dados de Treino e de Teste

In [ ]:
from sklearn.model_selection import train_test_split  # dividir/repartir teste e treino


A função train_test_split divide os dados (entradas `x` e saídas `y`) em **dados de treinamento** e **dados de teste**, retornando quatro resultados:
*   entradas e saídas de treinamento
*   entradas e saídas de teste

A proporção dos dados de teste é indicada pelo parâmetro `test_size`.



In [ ]:
# chamando a função train_test_split
# e recebendo seus quatro resultados em quatro variáveis
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=8, stratify=y)

Quantos exemplos de treinamento foram escolhidos?

In [ ]:
len(x_train)  # deve ser igual ao len(y_train)

In [ ]:
x_train[0]

E quantos exemplos de teste?

In [ ]:
len(x_test)  # tem que ser igual a len(y_test)

In [ ]:
x_test[0]

## 4 - Treinando uma Árvore de Decisão

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree

É bem simples, basta iniciar e chamar `fit` passando as entradas e as saídas corretas correspondentes.

In [ ]:
tree_model = DecisionTreeClassifier()

In [ ]:
tree_model.fit(x_train, y_train)

### 4.1 - Usando o Modelo Treinado

Código para testar só um exemplo de entrada específica.

Primeiro, vamos ver os valores dos atributos na **sexta linha** (índice 5) dos exemplos de teste.

In [ ]:
print(x_test[5])

E a classe de saída dele informada no conjunto de dados:

In [ ]:
print(y_test[5])

Agora, vamos preparar um exemplo parecido (quase igual), passar para o modelo e imprimir o resultado:

In [ ]:
entrada = [ [4.4, 1.4] ]
saida_prevista = tree_model.predict( entrada )
print(saida_prevista)

Para mandar o modelo fazer a previsão de todas as entradas de teste (`x_test`):

In [ ]:
y_previsto_arvore = tree_model.predict(x_test)
y_previsto_arvore

### 4.2 - Comparando com as Saídas Reais

Vamos comparar com as saídas corretas dos dados de teste.

In [ ]:
y_test

É fácil fazer a comparação elemento por elemento, para ver se nosso modelo acertou tudo:

In [ ]:
sum(y_previsto_arvore == y_test) / len(y_test)

### 4.3 Visualizando o Espaço de Entradas

Abaixo, vamos mostrar uma figura que apresentada todos os exemplos (de treinamento e de teste) como pontos assim:
- os dois atributos de entrada são usados como coordenadas do ponto
- as classes (0, 1 e 2) definem o desenho geométrico de cada ponto
- as amostras de teste têm, a mais, um círculo em volta dela
- as cores de fundo mostram as regiões que o modelo aprendeu como sendo a região dos pontos de cada classe

In [ ]:
plt.figure(figsize=(10,8))
plot_decision_regions(x_train, y_train, tree_model, x_test, y_test)

plt.xlabel('Petal length [cm]')
plt.ylabel('Petal width [cm]')
plt.legend(loc='upper left')
plt.tight_layout()

#plt.savefig('espaco_arvore_decisao', dpi=300)
plt.show()

### 4.4 Visualizando o Modelo

As árvores de decisão são um dos poucos modelos que oferecem uma opção de vizualização.

Vamos ver a nossa árvore treinada? (Para entender, veja o material).

In [ ]:
feature_names = ['Petal length', 'Petal width']

plt.figure(figsize=(14,12))
plot_tree(tree_model, feature_names=feature_names, filled=True)

#plt.savefig('arvore_decisao_iris.pdf')
plt.show()

## 5 - Treinando uma Random Forest (Floresta Aleatória)

Todo o processo para treinar outros modelos de classificação vai ser muito parecido. Só muda o nome dele e em qual submódulo do sklearn ele está (para você importar de lá).

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
floresta = RandomForestClassifier()

In [ ]:
floresta.fit(x_train, y_train)

In [ ]:
y_previsto_floresta = floresta.predict(x_test)

In [ ]:
y_previsto_floresta

In [ ]:
sum(y_previsto_floresta == y_test) / len(y_test)

In [ ]:
plt.figure(figsize=(10,8))
plot_decision_regions(x_train, y_train, floresta, x_test, y_test)

plt.xlabel('Petal length')
plt.ylabel('Petal width')
plt.legend(loc='upper left')
plt.tight_layout()

plt.show()

## 6 - Treinando um Modelo KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier()

In [ ]:
knn.fit(x_train, y_train)

In [ ]:
y_previsto_knn = knn.predict(x_test)

In [ ]:
sum(y_previsto_knn == y_test) / len(y_test)

In [ ]:
plt.figure(figsize=(10,8))
plot_decision_regions(x_train, y_train, knn, x_test, y_test)

plt.xlabel('Petal length')
plt.ylabel('Petal width')
plt.legend(loc='upper left')
plt.tight_layout()

plt.show()

In [ ]:
# outro modelo para você testar
#from sklearn.linear_model import LogisticRegression
#logistic_regr = LogisticRegression()

## 7 - Avaliando os Modelos

Esta parte é relativamente complexa. Veja mais detalhes na aula (material ou vídeo).

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

### 7.1 - Matriz de Confusão

In [ ]:
print(confusion_matrix(y_test, y_previsto_arvore))

In [ ]:
print(confusion_matrix(y_test, y_previsto_floresta))

In [ ]:
print(confusion_matrix(y_test, y_previsto_knn))

### 7.2 - Relatórios com Métricas Diversas

In [ ]:
print(classification_report(y_test, y_previsto_arvore))

In [ ]:
print(classification_report(y_test, y_previsto_floresta))

In [ ]:
print(classification_report(y_test, y_previsto_knn))

In [ ]:
#print(classification_report(y_test, y_previsto_knn, target_names=iris.target_names))

## 8 - Treinando e Testando Outro Modelo em Poucas Linhas

Nas linhas a seguir, vamos treinar um modelo chamado **regressão logística** (*logistic regression*). Apesar do nome, ele não faz regressão, faz classificação.

Note que, após importar e criar o modelo, o processo de treinar, fazer previsões e testar é o mesmo de antes, e pode ser feito em poucas linhas.

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logistic = LogisticRegression()

logistic.fit(x_train, y_train)

In [ ]:
y_previsto_logi = logistic.predict(x_test)
y_previsto_logi

In [ ]:
print(classification_report(y_test, y_previsto_logi))

# Referências

- Livro **"Machine Learning with Pytorch and Scikit-Learn"** (Raschka, Liu e Mirjalili, 2022)